In [133]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import plotly.express as px
import plotly.graph_objects as go
import json

CENTER_BARCELONA = {"lat": 41.3851, "lon": 2.1734}


def convert_wkt_to_geometry(df: pd.DataFrame, wkt_column: str) -> gpd.GeoDataFrame:
    # Convert the GEOM_WKT column to geometry
    df["geometry"] = df[wkt_column].apply(wkt.loads)

    # Convert the DataFrame to a GeoDataFrame
    return gpd.GeoDataFrame(df.drop(wkt_column, axis="columns"), geometry="geometry")

In [134]:
vage_df = pd.read_csv("data/age_of_vehicle/2023/2023_Antiguitat_tipus_vehicle.csv")

# vage_df['Nom_Districte'] = vage_df['Nom_Districte'].replace('Sants-MontjuÃ¯c', 'Sants-Montjuïc')
# vage_df['Nom_Districte'] = vage_df['Nom_Districte'].replace('SarriÃ -St. Gervasi', 'Sarrià-Sant Gervasi')
# vage_df['Nom_Districte'] = vage_df['Nom_Districte'].replace('GrÃ cia', 'Gràcia')
# vage_df['Nom_Districte'] = vage_df['Nom_Districte'].replace('Horta-GuinardÃ³', 'Horta-Guinardó')
# vage_df['Nom_Districte'] = vage_df['Nom_Districte'].replace('Sant MartÃ­', 'Sant Martí')

# vage_df.to_csv('data/age_of_vehicle/2023/2023_Antiguitat_tipus_vehicle.csv', index=False)

In [135]:
total_vehicles_per_district = (
    vage_df.groupby(["Nom_Districte"]).Nombre.sum().reset_index()
)
total_vehicles_per_district.columns = ["Nom_Districte", "Total_Vehicles"]

old_vehicles_per_district = (
    vage_df[vage_df.Antiguitat == "MÃ©s de 20 anys"]
    .groupby(["Nom_Districte"])
    .Nombre.sum()
    .reset_index()
)

old_vehicles_per_district.columns = ["Nom_Districte", "Vehicles_20_Any"]

In [136]:
merged = total_vehicles_per_district.merge(
    old_vehicles_per_district, on="Nom_Districte", how="left"
)
merged["Percentage"] = (merged["Vehicles_20_Any"] / merged["Total_Vehicles"]) * 100
merged["Percentage"] = merged["Percentage"].map("{:,.2f}".format)
merged = merged[merged.Nom_Districte != "No consta"]
merged["Percentage"] = merged["Percentage"].astype(float)

In [137]:
district_df = pd.read_csv("./data/district_zone/BarcelonaCiutat_Districtes.csv")

gdf = convert_wkt_to_geometry(district_df, "geometria_wgs84")
gdf = gdf.rename(columns={"nom_districte": "Nom_Districte"})

gdf_age = gdf.merge(merged, on="Nom_Districte", how="left")
gdf_age_json = json.loads(gdf_age.to_json())

In [ ]:
gdf_age.Vehicles_20_Any.sum() / gdf_age.Total_Vehicles.sum() * 100

In [139]:
# fig = px.choropleth_mapbox(
#     gdf_merged,
#     geojson=gdf_merged,
#     locations=gdf_merged.index,
#     color="Percentage",
#     color_continuous_scale="OrRd",
#     mapbox_style="carto-positron",
#     center=CENTER_BARCELONA,
#     zoom=10,
#     hover_name="Nom_Districte",
#     hover_data={
#         "Percentage": True,
#     },
#     title="Percentage of vehicles older than 20 years per district",
# )

# fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})

# fig.show()

In [ ]:
fig = go.Figure(
    go.Choroplethmapbox(
        geojson=gdf_age_json,
        locations=gdf_age.index,
        z=gdf_age["Percentage"].astype(float),
        colorscale="OrRd",
        marker_opacity=0.7,
        marker_line_width=0.5,
        colorbar_title="Pourcentage (%)",
        text=gdf_age["Nom_Districte"]
        + "<br>Pourcentage: "
        + gdf_age["Percentage"].astype(str)
        + "%",
    )
)

fig.update_layout(
    title_text="Pourcentage de véhicules de plus de 20 ans par district",
    mapbox_zoom=10,
    mapbox_center=CENTER_BARCELONA,
    margin={"r": 0, "t": 40, "l": 0, "b": 0},
    mapbox_style="carto-positron"
)

fig.update_layout(
    mapbox_style="carto-darkmatter",
    paper_bgcolor="black",
    plot_bgcolor="black",
    font=dict(color="white")  
)

fig.show()

---

In [141]:
pop_df = pd.read_csv("data/population/2023/2023_pad_mdbas.csv")

# pop_df['Nom_Districte'] = pop_df['Nom_Districte'].replace('Sants-MontjuÃ¯c', 'Sants-Montjuïc')
# pop_df['Nom_Districte'] = pop_df['Nom_Districte'].replace('SarriÃ -Sant Gervasi', 'Sarrià-Sant Gervasi')
# pop_df['Nom_Districte'] = pop_df['Nom_Districte'].replace('GrÃ cia', 'Gràcia')
# pop_df['Nom_Districte'] = pop_df['Nom_Districte'].replace('Horta-GuinardÃ³', 'Horta-Guinardó')
# pop_df['Nom_Districte'] = pop_df['Nom_Districte'].replace('Sant MartÃ­', 'Sant Martí')

# pop_df.to_csv('data/population/2023/2023_pad_mdbas.csv', index=False)

In [142]:
pop_per_district = pop_df.groupby(["Nom_Districte"]).Valor.sum().reset_index()
pop_per_district.columns = ["Nom_Districte", "Population"]

In [143]:
vtype_df = pd.read_csv(
    "data/type_of_vehicle/2023/2023_Parc_vehicles_tipus_propulsio.csv"
)

# vtype_df['Nom_Districte'] = vtype_df['Nom_Districte'].replace('Sants-MontjuÃ¯c', 'Sants-Montjuïc')
# vtype_df['Nom_Districte'] = vtype_df['Nom_Districte'].replace('SarriÃ -St. Gervasi', 'Sarrià-Sant Gervasi')
# vtype_df['Nom_Districte'] = vtype_df['Nom_Districte'].replace('GrÃ cia', 'Gràcia')
# vtype_df['Nom_Districte'] = vtype_df['Nom_Districte'].replace('Horta-GuinardÃ³', 'Horta-Guinardó')
# vtype_df['Nom_Districte'] = vtype_df['Nom_Districte'].replace('Sant MartÃ­', 'Sant Martí')
# vtype_df['Tipus_Propulsio'] = vtype_df['Tipus_Propulsio'].replace('ElÃ¨ctrica', 'Elèctrica')
# df['Tipus_Propulsio'] = vtype_df['Tipus_Propulsio'].replace('HÃ­brid', 'Híbrid')

# vtype_df.to_csv('data/type_of_vehicle/2023/2023_Parc_vehicles_tipus_propulsio.csv', index=False)

In [144]:
vehuicles_per_district = vtype_df.groupby(["Nom_Districte"]).Nombre.sum().reset_index()
vehuicles_per_district.columns = ["Nom_Districte", "Total_Vehicles"]

In [145]:
merged = vehuicles_per_district.merge(pop_per_district, on="Nom_Districte", how="left")
merged["Vehicles_Per_100"] = (merged["Total_Vehicles"] / merged["Population"]) * 100
merged = merged[merged.Nom_Districte != "No consta"]
merged["Vehicles_Per_100"] = merged["Vehicles_Per_100"].map("{:,.2f}".format)

In [ ]:
gdf_merged = gdf.merge(merged, on="Nom_Districte", how="left")

geojson = json.loads(gdf_merged.to_json())

fig = go.Figure(
    go.Choroplethmapbox(
        geojson=geojson,
        locations=gdf_merged.index,
        z=gdf_merged["Vehicles_Per_100"].astype(float),
        colorscale="OrRd",
        marker_opacity=0.7,
        marker_line_width=0.5,
        colorbar_title="Vehicles Per 100 habitants (%)",
        text=gdf_merged["Nom_Districte"]
        + "<br>Vehicles Per 100: "
        + gdf_merged["Vehicles_Per_100"].astype(str)
        + "‰",
    )
)

fig.update_layout(
    title_text="Vehicles Per 100 habitant per district",
    mapbox_style="carto-positron",
    mapbox_zoom=10,
    mapbox_center=CENTER_BARCELONA,
    margin={"r": 0, "t": 40, "l": 0, "b": 0},
)

fig.show()

---

In [147]:
# number of electric or hybrid vehicles per district
green_vehicles_per_district = (
    vtype_df[
        (vtype_df.Tipus_Propulsio == "Elèctrica")
        | (vtype_df.Tipus_Propulsio == "Híbrid")
    ]
    .groupby(["Nom_Districte"])
    .Nombre.sum()
    .reset_index()
)
green_vehicles_per_district.columns = ["Nom_Districte", "Green_Vehicles"]

In [148]:
merged = vehuicles_per_district.merge(
    green_vehicles_per_district, on="Nom_Districte", how="left"
)
merged["Percentage"] = (merged["Green_Vehicles"] / merged["Total_Vehicles"]) * 100
merged["Percentage"] = merged["Percentage"].map("{:,.2f}".format)

In [ ]:
gdf_type = gdf.merge(merged, on="Nom_Districte", how="left")

gdf_type_json = json.loads(gdf_type.to_json())

fig = go.Figure(
    go.Choroplethmapbox(
        geojson=gdf_type_json,
        locations=gdf_type.index,
        z=gdf_type["Percentage"].astype(float),
        colorscale="greens",
        marker_opacity=0.7,
        marker_line_width=0.5,
        colorbar_title="Percentage (%)",
        text=gdf_type["Nom_Districte"]
        + "<br>Percentage: "
        + gdf_type["Percentage"].astype(str)
        + "%",
    )
)

fig.update_layout(
    title_text="Percentage of green vehicles per district",
    mapbox_style="carto-positron",
    mapbox_zoom=10.5,
    mapbox_center=CENTER_BARCELONA,
    margin={"r": 0, "t": 40, "l": 0, "b": 0},
)

fig.show()

In [150]:
# Unitats administratives de la ciutat de Barcelona

In [151]:
gdf_age = gdf_age.rename(columns={"Percentage": "Age_Percentage"})
gdf_type = gdf_type.rename(columns={"Percentage": "Green_Percentage"})

In [152]:
gdf_kmean = gdf_age.merge(gdf_type, on="Nom_Districte", how="left")
gdf_kmean = gdf_kmean[["Nom_Districte", "Age_Percentage", "Green_Percentage"]]


In [153]:
# K-means clustering with k = 3
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=0).fit(gdf_kmean[["Age_Percentage", "Green_Percentage"]])

gdf_kmean["Cluster"] = kmeans.labels_

# Merge cluster data
gdf_kmean = gdf.merge(gdf_kmean, on="Nom_Districte", how="left")

# Convert to JSON
gdf_kmean_json = json.loads(gdf_kmean.to_json())

In [ ]:
cluster_colors = {
    "0": "#1f77b4",  
    "1": "#ff7f0e",  
    "2": "#2ca02c",  
}

gdf_kmean["Cluster"] = gdf_kmean["Cluster"].astype(str)

gdf_kmean["Color"] = gdf_kmean["Cluster"].map(cluster_colors)

fig = go.Figure()

for cluster, color in cluster_colors.items():
    cluster_data = gdf_kmean[gdf_kmean["Cluster"] == cluster]

    fig.add_trace(
        go.Choroplethmapbox(
            geojson=gdf_kmean_json,
            locations=cluster_data.index,
            z=[int(cluster)] * len(cluster_data),
            colorscale=[[0, color], [1, color]],
            marker_opacity=0.7,
            marker_line_width=0.5,
            showscale=False,
        )
    )

for cluster, color in cluster_colors.items():
    fig.add_trace(
        go.Scattermapbox(
            lat=[None], 
            lon=[None],  
            mode="markers",
            marker=dict(size=15, color=color),
            name=f"Cluster {cluster}",
        )
    )

fig.update_layout(
    title_text="K-means clustering des districts",
    mapbox_style="carto-positron",
    mapbox_zoom=10.5,
    mapbox_center=CENTER_BARCELONA,
    margin={"r": 0, "t": 40, "l": 0, "b": 0},
    legend=dict(title="Clusters"),
)

fig.show()


In [202]:
vage_df = pd.read_csv("data/age_of_vehicle/2023/2023_Antiguitat_tipus_vehicle.csv")

In [203]:
vage_df["Antiguitat"] = vage_df["Antiguitat"].replace("Menys d'un any d'antiguitat", "Moins d'un an")
vage_df["Antiguitat"] = vage_df["Antiguitat"].replace("D'11 a 20 anys", "De 11 a 20 ans")
vage_df["Antiguitat"] = vage_df["Antiguitat"].replace("MÃ©s de 20 anys", "Plus de 20 ans")
vage_df["Antiguitat"] = vage_df["Antiguitat"].replace("No consta", "Non spécifié")
vage_df["Antiguitat"] = vage_df["Antiguitat"].replace("1 any", "1 an")
vage_df["Antiguitat"] = vage_df["Antiguitat"].replace("2 anys", "2 ans")
vage_df["Antiguitat"] = vage_df["Antiguitat"].replace("3 anys", "3 ans")
vage_df["Antiguitat"] = vage_df["Antiguitat"].replace("4 anys", "4 ans")
vage_df["Antiguitat"] = vage_df["Antiguitat"].replace("5 anys", "5 ans")
vage_df["Antiguitat"] = vage_df["Antiguitat"].replace("6 anys", "6 ans")
vage_df["Antiguitat"] = vage_df["Antiguitat"].replace("7 anys", "7 ans")
vage_df["Antiguitat"] = vage_df["Antiguitat"].replace("8 anys", "8 ans")
vage_df["Antiguitat"] = vage_df["Antiguitat"].replace("9 anys", "9 ans")
vage_df["Antiguitat"] = vage_df["Antiguitat"].replace("10 anys", "10 ans")
vage_df = vage_df[vage_df.Antiguitat != "Non spécifié"]

vage_df.to_csv('data/age_of_vehicle/2023/2023_Antiguitat_tipus_vehicle2.csv', index=False)

In [204]:
print(vage_df.Antiguitat.unique())

["Moins d'un an" '1 an' '2 ans' '3 ans' '4 ans' '5 ans' '6 ans' '7 ans'
 '8 ans' '9 ans' '10 ans' 'De 11 a 20 ans' 'Plus de 20 ans']


In [205]:
df_grouped = vage_df[['Antiguitat', 'Nombre']].groupby("Antiguitat", as_index=False).sum()

In [206]:
fig = px.pie(
    df_grouped, 
    names="Antiguitat", 
    values="Nombre", 
    title="Répartition des véhicules par âge",
    color_discrete_sequence=px.colors.sequential.Reds
)
fig.update_traces(textinfo="percent+label")
fig.show()